In [8]:
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

import requests
from dotenv import load_dotenv
import os
load_dotenv()

True

In [9]:
gpt_turbo = OpenAI(model_name="gpt-3.5-turbo-instruct")

In [10]:
url = 'https://public-api.meteofrance.fr/public/DPVigilance/v1/textesvigilance/encours'
headers = {
    'accept': '*/*',
    'apikey': os.environ['VIGILENCE_API_KEY']
}
response = requests.get(url, headers=headers)
data = response.json()

In [11]:
def extract_text_items(data):
    text_items = []
    infos = [
        region for region in data['product']['text_bloc_items'] 
        if len(region['bloc_items']) != 0 and region['bloc_id']=='BULLETIN_DEPARTEMENTAL'
    ]
    for info in infos:
        for bloc_item in info.get('bloc_items', []):
            for text_item in bloc_item.get('text_items', []):
                for subdivision in text_item.get('term_items', []):
                    for subdivision_text in subdivision.get('subdivision_text', []):
                        text_items.append("".join(subdivision_text.get('text', '')))
    return text_items

In [12]:
infos = list(set(extract_text_items(data)))

In [56]:
template = """
Write a concise summary in french of the following:
"{informations}"
CONCISE SUMMARY:
"""

In [57]:
prompt = PromptTemplate.from_template(template)


In [58]:
chain = (
    {'informations':RunnablePassthrough()} 
    | prompt 
    | gpt_turbo
)

In [59]:
infos

['Tempête tardive pour la saison, sans caractère exceptionnel, engendrant des vents violents, de fortes vagues et un risque de submersion côtière.',
 'Mardi, "PIERRICK" circule sur l\'Angleterre et se décale en Mer du Nord.Dans sa partie sud, les vents de secteur Ouest sont forts en Manche et génèrent de fortes vagues.Dans un contexte de grandes marées, la conjonction de hauts niveaux d\'eau et de fortes vagues induit un risque marqué de "vagues-submersion" autour de la marée haute de mardi matin (coefficient 112) pour les départements du Finistère, des Côtes-d\'Armor, de l\'Ille-et-Vilaine et de l\'ouest de la Manche, placés en vigilance "vagues-submersion" de niveau orange. Les heures de pleines mers sont les suivantes : Roscoff 07h25, Paimpol 08h26, Saint-Malo 08h31, Granville 08h40.Dans le même temps, une houle de Nord-Ouest se propage dans le golfe de Gascogne. Tous les départements de la façade atlantique sont alors placés en vigilance "vagues-submersion" de niveau jaune pour mar

In [60]:
answer = chain.invoke(infos)
print(answer)

Tempête tardive pour la saison avec vents violents et risque de submersion côtière. La dépression PIERRICK circule sur l'Angleterre et se décale en Mer du Nord, générant des vents forts et de fortes vagues sur les côtes de Manche et Atlantique. Les départements de Finistère, Côtes-d'Armor, Ille-et-Vilaine, ouest de la Manche et façade atlantique sont en vigilance pour les vagues-submersion. La conjonction de grandes marées et de fortes vagues nécessite une vigilance particulière. Passage en vigilance orange pour vents violents sur le Finistère. Incertitude sur le positionnement exact du creusement lié à PIERRICK.
